# HW4 - Knowledge engineering

### Steps for assignment 
- Dowload dataset from kirc-page. It is part of IMDB data
- Apply NPL techniques to the plot data, keywords, synopsis (stem, lemmatization, rm stop-words, 
- Create two term-movie matrixes:
- - Nr 1: keywords
- - Nr2: terms from synopsis and plotsummary
- Measure similarity between movies by cosine similarity


### Characteristics about the data:
- IMDB data
- Ratings on the scale from 1 to 10
- Content about the movies, likely key-value pairs
- 2k movies
- separated into: Movie, plotsummary and keywords
- Separator = '|'


#### Informal description - Movie
- Idmovie: id
- Ranking: a box-office record
- name: name of movie
- Country: country movie was made
- ReleaseYear
- Synosis: 
- Certificate: film rate of movie
- Runtime

#### Informal deciption - Plotsummary:
- idplotsummary
- content
- movie_idmovie

#### informal desc - Keyword
- Idkeyword
- Keyword
- Movie_idmovie: list of the movie id's with this keyword


### Extra info prom prof: 
- Can decide myself how many keywords i would like to use
- Using too many keywords will decrease accuracy
- Using to few keywords will decrease recall

## Questions:
### Q1
Using the first matrix (Keyword), find top 10 similar movies to "titanic". Print as: (movieName1,movie_id1, distance_score) and so on. Send screen of code
### Q2
Find top 10 movies for each of the first top 10 movies, by second matrix (synopsis and plotSummary). Same print format. Send screenshot of code
### Q3
Which recommendation results look more reasonable and why? provide academic references if possible (Max 1 page incl references for this question)

- The keyword method cannot use frequency or counting to select which terms are more important. Therefore it is difficult to select a subset of keywords from the keywords in better ways than random selection 
- Further details in docx

## Deliverable:
Word doc of q3 plus screens of q1 and q2, **file name: HW4_student-ID_Surname.docx**  
Code file: Well commented and executable, **Submit python file as well as the jupyter notebook file** 

In [0]:
'''Setup of environment'''
!pip install --user nltk
!pip install --user rake_nltk

import nltk
nltk.download('all')

import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


# Requirements: 
- Run in google colab with filepath to csv files specified, or changing path into the path where files are stored locally
- Two cases of separator sign "|" removed from synopsis of movie "Notorious" in movie.csv
- Log in to Drive following the link authentication below, using googles drive class from the colab library

In [2]:
mountPoint = '/content/gdrive'
''' Uncomment for usage of coogle colab

from google.colab import drive
 
drive.mount(mountPoint)
'''
# Mounting of google drive in order to fetch the datasets

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
dataset = {}

#Edit the full drive path to where you have decided to store the csv-files if using google drive
fullDrivePath = mountPoint+'/My Drive/Colab Notebooks/Knowledge engineering dataset/'

# check that filenames are correct for your files
filenames = ['plotsummary.csv','movie.csv','keywords.csv']

nlpColumns = {'movie':['synopsis'],'plotsummary':['contents'],'keywords':['keyword']} #Columns to perform cleaning
stringColumns = nlpColumns.copy()  #Columns to change type into string
stringColumns['movie'] = ['synopsis','country','name','runtime'] 

for i in range (3):
  # Alter path accordingly
  #path = fullDrivePath+filenames[i] # using gDrive
  path = filenames[i] # local files in same folder as .ipynb file
  
  key = filenames[i][0:-4]
  dataset[key] = pd.read_csv(path, sep='|', encoding='ascii', header=0)
  dataset[key].columns = dataset[key].columns.str.strip() # Removing uneccecary whitespaces in the dataset

for key, value in stringColumns.items():
  dataset[key][value] = dataset[key][value].astype(str)
  
for key in dataset:
  print (key+"\'s columns are: \n" + str(dataset[key].columns.tolist())+'\n')

plotsummary's columns are: 
['plot_id', 'movie_id', 'contents']

movie's columns are: 
['movie_id', 'ranking', 'name', 'country', 'release_year', 'certificate', 'runtime', 'synopsis']

keywords's columns are: 
['keyword_id', 'keyword', 'movie_id']



In [0]:
# NLP method
from nltk.corpus import stopwords

# Assumption: results can be improved further by joining the first and last names in the synopsis,
#     because similarity will only be detected between the same actor/director, not between actors with the same first name
'''
def joinNames(text):
  text = text.split()
  for i in range(1,len(text)):
    if text[i-1][0].isupper():
      if text[i][0].isupper():
        print(text[i-1])
        #text = text[0:i-2]+[text[i
'''

def cleanText(text):
  #rm spaces between first and last names
  #text = " ".join(joinNames(text))
  
  # split into words/tokens
  from nltk import word_tokenize
  tokens = word_tokenize(text)

  # set to lower case
  tokens = [word.lower() for word in tokens]

  # stem words
  from nltk.stem.porter import PorterStemmer
  porter = PorterStemmer()
  stemmed = [porter.stem(word) for word in tokens]

  # rm puctuation
  import string
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in stemmed]

  # remove non-aplhabetic charachters
  words = [word for word in stripped if word.isalpha()]
  
  # remove stop words using nltk's stop word list
  stop_words_general = stopwords.words('english')
  cleanedText = [word for word in words if not word in stop_words_general]
  
  cleanedText = " ".join(cleanedText)
  
  return cleanedText

In [5]:
# NLP process the columns in nlpColumns
for key in dataset:
  print(key+" is being cleaned ", end = '')
  dataset[key][nlpColumns[key]] = dataset[key][nlpColumns[key]].applymap(lambda s: cleanText(s))
  print("- "+key+" is done")

dataset['movie'].head()

plotsummary is being cleaned - plotsummary is done
movie is being cleaned - movie is done
keywords is being cleaned - keywords is done


,movie_id,ranking,name,country,release_year,certificate,runtime,synopsis
0,tt0499549,1,Avatar,USA,2009,12,162min,open scene camera sweep high across treelin lu...
1,tt0120338,2,Titanic,USA,1997,15,194min,brock lovett bill paxton hi team treasur hunte...
2,tt0848228,3,The Avengers,USA,2012,12,143min,shield director nick furi samuel l jackson arr...
3,tt0468569,4,The Dark Knight,USA,2008,15,152min,movi begin gang men clown mask break bank mob ...
4,tt0120915,5,Star Wars: Episode I - The Phantom Menace,USA,1999,All,136min,open crawl reveal trade feder ha blockad plane...


In [6]:
# concatnate plotsummaries for the same movie
dataset['plotsummary_joinedSummaries'] = dataset['plotsummary'].groupby(['movie_id'])['contents'].apply(lambda x: ' '.join(x)).reset_index()

# merge plotsummary and movie dataframe
dataset['synopsis_and_summary'] = pd.merge(dataset['movie'],dataset['plotsummary_joinedSummaries'][['movie_id','contents']],how='inner', on='movie_id')

dataset['synopsis_and_summary'].head()

,movie_id,ranking,name,country,release_year,certificate,runtime,synopsis,contents
0,tt0499549,1,Avatar,USA,2009,12,162min,open scene camera sweep high across treelin lu...,hi brother kill robberi parapleg marin jake su...
1,tt0120338,2,Titanic,USA,1997,15,194min,brock lovett bill paxton hi team treasur hunte...,year later woman name rose dewitt bukat tell s...
2,tt0848228,3,The Avengers,USA,2012,12,143min,shield director nick furi samuel l jackson arr...,nick furi director shield intern peac keep age...
3,tt0468569,4,The Dark Knight,USA,2008,15,152min,movi begin gang men clown mask break bank mob ...,batman rais stake hi war crime help lieuten ji...
4,tt0120915,5,Star Wars: Episode I - The Phantom Menace,USA,1999,All,136min,open crawl reveal trade feder ha blockad plane...,evil trade feder led nute gunray plan take pea...


In [7]:
from heapq import nlargest
import time
#Making bag of words for synopsis and plotsummary
dataset['synopsis_and_summary']['bag_of_words'] = ""

def applyRake(plot,summary):
  r = Rake()
  # extracting the words by passing the text
  r.extract_keywords_from_text(plot+summary)
  
  # getting the dictionary whith key words as keys and their scores as values
  key_words_dict_scores = r.get_word_degrees()
  
  #nHighestScoreKeys = nlargest(100, key_words_dict_scores, key = key_words_dict_scores.get) 
  nHighestScoreKeys = get_n_largest_values_g(key_words_dict_scores, 120) #choose a number of keywords between 50 and 200, as a rule of thumb discussed in lecture. 150 matches sample amount per movie in keywords.csv
  
  return " ".join(nHighestScoreKeys)

def get_n_largest_values_g(dictionary, n):
    largest_entries = sorted(
        dictionary.items(), key=lambda t: t[1], reverse=True)[:n]
    return [key for key, value in largest_entries]

start = time.time()
dataset['synopsis_and_summary']['bag_of_words'] = dataset['synopsis_and_summary'].apply(lambda row: applyRake(row['synopsis'],row['contents']), axis = 1)
end = time.time()
print(end-start)
# With get_n_largest_values_g: entire operation takes ~570 seconds on colab vm
# With heapq nlargest: entire operation takes ~600 seconds on colab vm

dataset['synopsis_and_summary'].head()

571.7368190288544


,movie_id,ranking,name,country,release_year,certificate,runtime,synopsis,contents,bag_of_words
0,tt0499549,1,Avatar,USA,2009,12,162min,open scene camera sweep high across treelin lu...,hi brother kill robberi parapleg marin jake su...,jake hi line break neytiri navi grace avatar c...
1,tt0120338,2,Titanic,USA,1997,15,194min,brock lovett bill paxton hi team treasur hunte...,year later woman name rose dewitt bukat tell s...,rose jack hi ship break line cal say tell ha l...
2,tt0848228,3,The Avengers,USA,2012,12,143min,shield director nick furi samuel l jackson arr...,nick furi director shield intern peac keep age...,loki furi hi tesseract earth thor shield hulk ...
3,tt0468569,4,The Dark Knight,USA,2008,15,152min,movi begin gang men clown mask break bank mob ...,batman rais stake hi war crime help lieuten ji...,batman hi joker dent harvey gordon break line ...
4,tt0120915,5,Star Wars: Episode I - The Phantom Menace,USA,1999,All,136min,open crawl reveal trade feder ha blockad plane...,evil trade feder led nute gunray plan take pea...,naboo jedi quigon feder anakin obiwan trade qu...


In [8]:
# Making bag of words based on keywords
dataset['keywords_grouped'] = dataset['keywords'].groupby(['movie_id'])['keyword'].apply(lambda x: ' '.join(x)).reset_index().rename(columns={'keyword':'bag_of_words'})
dataset['keywords_grouped'] = pd.merge(dataset['movie'], dataset['keywords_grouped'], how='inner', on='movie_id')

dataset['keywords_grouped'].head()

,movie_id,ranking,name,country,release_year,certificate,runtime,synopsis,bag_of_words
0,tt0499549,1,Avatar,USA,2009,12,162min,open scene camera sweep high across treelin lu...,parapleg nativ colonel marin battl soldier epi...
1,tt0120338,2,Titanic,USA,1997,15,194min,brock lovett bill paxton hi team treasur hunte...,titan ship love diamond artist draw iceberg su...
2,tt0848228,3,The Avengers,USA,2012,12,143min,shield director nick furi samuel l jackson arr...,shield tesseract captain thor captainamerica i...
3,tt0468569,4,The Dark Knight,USA,2008,15,152min,movi begin gang men clown mask break bank mob ...,gotham crimin vigilant criminalmastermind dist...
4,tt0120915,5,Star Wars: Episode I - The Phantom Menace,USA,1999,All,136min,open crawl reveal trade feder ha blockad plane...,planet queen warrior apprentic alien lightsab ...


In [0]:

def recommendationsBody(title, dataframe, cosine_sim, titleColumnName):
    # creating a Series for the movie titles so they are associated to an ordered numerical
    # list to match movie name to similarity index
    indices = pd.Series(dataframe[titleColumnName])
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(((dataframe[titleColumnName])[i],(dataframe['movie_id'])[i],score_series[i]))
        
    return recommended_movies
  
  
# function that takes in movie title, dataframe and two columnNames 
# as input and returns the top 10 recommended movies per movie entered
def recommendations(movies, dataframe, keywordColumnName, titleColumnName):
  # instantiating and generating the count matrix
  count = CountVectorizer()
  count_matrix = count.fit_transform(dataframe[keywordColumnName])

  # generating the cosine similarity matrix
  cosine_sim = cosine_similarity(count_matrix, count_matrix)
  
  if isinstance(movies, list): # case of list for multiple movie input
    topTenMovies_for_listOfMovies = {}
    for movie in movies:
      topTenMovies_for_listOfMovies[movie] = recommendationsBody(movie, dataframe, cosine_sim, titleColumnName)
    return topTenMovies_for_listOfMovies
  
  elif isinstance(movies, str): # case of single input
    return recommendationsBody(movies,dataframe,cosine_sim, titleColumnName)
  
  else:
    raise Exception('movies should be a string for a movieTitle in the dataframe, or a list of movieTitles in the dataframe')

## Using keyword matrix to find top 10 similar movies to the Top 10 movies

In [10]:
listOfTopTenMovies = dataset['movie'].iloc[0:10][['name']].values.tolist()
listOfTopTenMovies = [movieSingleton[0] for movieSingleton in listOfTopTenMovies]
topTenKeyword = recommendations(listOfTopTenMovies, dataset['keywords_grouped'],'bag_of_words', 'name')
topTenKeyword

{'Avatar': [('Predators', 'tt1424381', 0.23662226312111664),
  ('The Last Samurai', 'tt0325710', 0.22180461419557174),
  ('Man of Steel', 'tt0770828', 0.22144100086073473),
  ('Dances with Wolves', 'tt0099348', 0.2124226547565469),
  ('Star Trek Into Darkness', 'tt1408101', 0.20981358185407048),
  ('Predator', 'tt0093773', 0.2042629579954974),
  ('Aliens', 'tt0090605', 0.20353162106401748),
  ('Underworld: Awakening', 'tt1496025', 0.19643781483510436),
  ('Prometheus', 'tt1446714', 0.19402985717067472),
  ('Alien: Resurrection', 'tt0118583', 0.19034674690672038)],
 'E.T. the Extra-Terrestrial': [('Home Alone',
   'tt0099785',
   0.17011927720118086),
  ('Cocoon', 'tt0088933', 0.15715902787873068),
  ('Close Encounters of the Third Kind', 'tt0075860', 0.14008295367013146),
  ('Edward Scissorhands', 'tt0099487', 0.14008295367013146),
  ('Paul', 'tt1092026', 0.12848904218751162),
  ('Halloween', 'tt0077651', 0.12749105166233532),
  ("Harry Potter and the Sorcerer's Stone", 'tt0241527', 0.

## Using synopsis+summary matrix to find top 10 similar movies to the top 10 movies

In [11]:
topTenSynopsis_and_summary = recommendations(listOfTopTenMovies,dataset['synopsis_and_summary'],'bag_of_words', 'name')
topTenSynopsis_and_summary

{'Avatar': [('The Chronicles of Narnia: The Lion, the Witch and the Wardrobe',
   'tt0363771',
   0.34166666666666634),
  ('Transformers', 'tt0418279', 0.33333333333333304),
  ('Gangs of New York', 'tt0217505', 0.33333333333333304),
  ('Terminator 2: Judgment Day', 'tt0103064', 0.33333333333333304),
  ('Rambo', 'tt0462499', 0.33333333333333304),
  ('The Wolverine', 'tt1430132', 0.32499999999999973),
  ('Inception', 'tt1375666', 0.32499999999999973),
  ('The Matrix Reloaded', 'tt0234215', 0.32499999999999973),
  ('X-Men: The Last Stand', 'tt0376994', 0.32499999999999973),
  ('The Matrix Revolutions', 'tt0242653', 0.31666666666666643)],
 'E.T. the Extra-Terrestrial': [("The 'Burbs",
   'tt0096734',
   0.4249999999999994),
  ('Changeling', 'tt0824747', 0.39999999999999947),
  ('Toy Story 3', 'tt0435761', 0.38333333333333286),
  ('Ratatouille', 'tt0382932', 0.36666666666666625),
  ('The Village', 'tt0368447', 0.36666666666666625),
  ('Four Christmases', 'tt0369436', 0.36666666666666625),
 

## Q3: Which recommendation results look more reasonable and why? 
 - In doc

Reference for how quick "apply" is compared to "iterrows": https://medium.com/@rtjeannier/pandas-101-cont-9d061cb73bfc

In [12]:
# Saving some test code for later experiments and learning

'''
for index, row in dataset['movie'].iterrows():
    
    plot = row['synopsis']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    
    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    #print(list(key_words_dict_scores.keys()))
    #print(row['key_words'])
    
    # assigning the key words to the new column for the corresponding movie
    dataset['movie'].loc[index, 'key_words'] = list(key_words_dict_scores.keys())
    #print(row['key_words'])
    #print("bab\n")
    #print(row)
    #print(dataset['movie'])
    
    if (index % 25) == 0:
      print(index)
      print(row)
      print(dataset['movie'])
'''
# dropping the Plot column
#dataset['movie'].drop(columns = ['synopsis'], inplace = True)

'\nfor index, row in dataset[\'movie\'].iterrows():\n    \n    plot = row[\'synopsis\']\n    \n    # instantiating Rake, by default it uses english stopwords from NLTK\n    # and discards all puntuation characters as well\n    r = Rake()\n\n    # extracting the words by passing the text\n    r.extract_keywords_from_text(plot)\n    \n    # getting the dictionary whith key words as keys and their scores as values\n    key_words_dict_scores = r.get_word_degrees()\n    #print(list(key_words_dict_scores.keys()))\n    #print(row[\'key_words\'])\n    \n    # assigning the key words to the new column for the corresponding movie\n    dataset[\'movie\'].loc[index, \'key_words\'] = list(key_words_dict_scores.keys())\n    #print(row[\'key_words\'])\n    #print("bab\n")\n    #print(row)\n    #print(dataset[\'movie\'])\n    \n    if (index % 25) == 0:\n      print(index)\n      print(row)\n      print(dataset[\'movie\'])\n'